<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab4: CTC(Connectionist Temporal Classification)
## contents
- why we use CTC 
---
## Why we use CTC
- 일반적인 speech recognition에서 우리는 dataset으로 오디오 클립과 transcript를 받게 됨(pair)
- 어떤 단어의 char가 audio와 alignment가 맞는지 알 수 없다.
- 이러한 alignment 없이, input audio와 output text 사이 mapping을 찾기 어렵
- 또한 사람들의 언어 사용은 다양하기 때문에 단일한 규칙으로 정의하기 어렵 - 같은 문장이어도 억양이 다름

## CTC
- input과 output 사이의 정확한 alignment가 labeling되어있는 data set이 필요하지 않지만 input에 대해서 ourput의 확률 값은 필요
- CTC는 둘 사이의 가능한 모든 alignment의 가능성을 합산하여 작용
- CTC는 조건부 확률분포, (x, y)의 pair dataset에 대해 각각의 x의 input step을 따라가면서 step-by-step으로 시간축으로 움직이며 single alignment의 확률 계산하고, 그중에서 validation alignment을 뽑아 marginalize를 진행
- time step별 확률분포를 얻고 input sequence의 context를 고려하기 위해서 RNN기반의 모델을 통해 학습
  - 하지만 RNN은 input sequence가 fixed-size splice 때 더 활용 용이

<img src="https://user-images.githubusercontent.com/7529838/33918455-c5aff7dc-dff6-11e7-8842-6a3b9d3bfcbd.png" width="500px" height="300px"></img>

<https://distill.pub/2017/ctc/>

- 문제점
1. 모든 x에 y가 할당되는 것은 옳지 않다. : 말은 띄어쓰기, 묵음이 존재 가능성, output 사이에 묵음이 존재할 수 있음을 가정해야 함
  - output에 토큰 \epsilon 을 추가, input과 같은 길이로 alignment 진행하고 y로 mapping하는 단계에서 \epsilon 제거

<img src="https://user-images.githubusercontent.com/7529838/33918530-3e0ce21c-dff7-11e7-8bc2-973e05c2c552.png" width="400px" height="250px"></img>

- 속성
1. 하나의 input에서 next input으로 진행하면서 output은 이전 step과 동일하거나 next output을 assign 
  - 순서가 중요
  - label 이미 있음: cat; catc -> c다음에는 c나 a 가 와 야 함 t는  label상 a가 와야하니까 안됨
2. x, y 정렬이 many-to-one 함수 : 하나 이상의 input이 output에 들어갈 수 있다. 하지만 반대는 성립X
<img src="https://distill.pub/2017/ctc/assets/valid_invalid_alignments.svg" width="400px" height="250px"></img>

- 순서

<img src="https://distill.pub/2017/ctc/assets/full_collapse_from_audio.svg" width="350px" height="400px"></img>

1. input sequence의 spectogram 으로 시작
2. input은 RNN계열 layer에 input으로 통과
3. network의 output으로 알파벳 단위의 전체 분포 반환 $p_{t}(\alpha | X)$, 이는 output인 {h, e, l, o, \epslion}의 각 input time step별의 확률분포 return
4. 각 time step별로 output을 가지고, 우리는 다른 sequence간의 확률을 구할 수 있다.
5. alignment에 대해 marginalize를 진행하면, 우리는 output의 확률분포를 구할 수 있다.

---
## code
[https://colab.research.google.com/drive/1Ttk_urX_dUvimaqlrhglC6ql3XNKKqSS?usp=sharing](https://colab.research.google.com/drive/1Ttk_urX_dUvimaqlrhglC6ql3XNKKqSS?usp=sharing)